In [1]:
import os
import time
import datetime
import argparse
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from transformers import BertModel, BertTokenizer
from keras.preprocessing.sequence import pad_sequences

import parallel

Using TensorFlow backend.


In [2]:
def gather_sentence_outputs(outputs):
    """
    'outputs' is a list of 'output' of each GPU. As a reminder, each 'output' is a 3-tuple where:
        - output[0] is the last_hidden_state, i.e a tensor of shape (batch_size, sequence_length, hidden_size).
        - output[1] is the pooler_output, i.e. a tensor of shape (batch_size, hidden_size) being the last layer hidden-state of the first token of the sequence (classification token).
        - output[2] are all hidden_states, i.e. a 13-tuple of torch tensors of shape (batch_size, sequence_length, hidden_size): 12 encoders-outputs + initial embedding outputs.
    """
    # Extract the last_hidden_state in each GPU output ('gathered' is a list of nb_gpu x torch tensors)
    gathered = [output[0] for output in outputs]
    
    # Concatenate the samples for that batch.
    gathered = torch.cat(gathered, dim=0)
    
    # Detach output and return.
    return gathered


In [3]:
sentences = ['Sentence A',
             'Sentence B',
             'Sentence C',
             'Sentence D',
             'Sentence E',
             'Sentence F',
             'Sentence G',
             'Sentence H',
             'Sentence I',
             'Sentence J',
             'Sentence K',
             'Sentence L',
             'Sentence M',
             'Sentence N',
             'Sentence O',
             'Sentence P',
             'Sentence Q',
             'Sentence R',
             'Sentence S',
             'Sentence T',
             'Sentence V',
             'Sentence W',
             'Sentence X',
             'Sentence Y',
             'Sentence AA',
             'Sentence AB',
             'Sentence AC',
             'Sentence AD',
             'Sentence AE',
             'Sentence AF',
             'Sentence AG',
             'Sentence AH'
]

In [4]:
model_name_or_path = '/raid/antoloui/Master-thesis/Code/_models/netbert/checkpoint-1027000/'
cache_dir = '/raid/antoloui/Master-thesis/Code/_cache'
batch_size = 16